In [14]:
from loader.daily_loader import load_daily_paper
from chater.chat import mapreduce
import os
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_nvidia_ai_endpoints import ChatNVIDIA
import faiss
import json

In [26]:
#初始化部分
os.environ["NVIDIA_API_KEY"] = "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"
model = ChatNVIDIA(model="ai-llama3-8b")#.bind(max_tokens=4096)
embedder = NVIDIAEmbeddings(model="ai-embed-qa-4")
cache = {}
try:
    cache = json.load(open("cache.json"))  # 读取用户有过哪些向量库   里面存的应该是tuple (vectorstore_path, initial_msg)
except:
    cache = {}
    json.dump(cache, open("cache.json", "w"))

这里先不给用户造自己的库的功能 只有daily chat和 awesome chat 两个功能  本地有个csv表格记录已经做了哪些库

In [19]:
from vec_generate.arxiv_generate import arxiv_generate

def daily_chat(date, cat, model, embedder, cache={}, verbose=False):

    '''
    接受日期 类别 模型和cache, 返回一个向量库路径和一个开场词
    如果用户指定verbose为True, 则当天的arxiv表格会保存在该库本地
    '''

    if date+cat in cache:
        return cache[date+cat]
    
    else:
        daily = load_daily_paper(date)

        if(len(daily) == 0):
            print("No papers found for this date and category, please retry")
            return None, None

        summary, detail = mapreduce(model, daily)
        if verbose:
            daily.to_csv(f"{date}_{cat}.csv")  # 保存当天的arxiv表格
        
        path_name = 'local_embed/'+date+cat
        arxiv_generate(detail['id'][:2], embedder, path_name)
        
        cache[date+cat] = (path_name, summary)
        json.dump(cache, open("cache.json", "w"))

        return path_name, summary
        
        

In [21]:
path, init_msg = daily_chat("20240707", "cs.CR", model, embedder, cache, verbose=True)

In [22]:
vecstores = [FAISS.load_local(folder_path=path, embeddings=embedder,allow_dangerous_deserialization=True)]

In [27]:
from vec_generate.arxiv_generate import aggregate_vstores

docstore = aggregate_vstores(vecstores, embedder)
print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

Constructed aggregate docstore with 223 chunks


In [29]:
from vec_generate.arxiv_generate import default_FAISS
convstore = default_FAISS(embedder)

In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from functools import partial
from langchain.document_transformers import LongContextReorder
from operator import itemgetter
from langchain_core.runnables.passthrough import RunnableAssign

    
def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = init_msg

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.Reply must more than 100 words)"
), ('user', '{input}')])

## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

## 将较长的文档重新排序到输出文本的中心， RunnableLambda在链中运行无参自定义函数 ，长上下文重排序（LongContextReorder）
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

retrieval_chain = (
    {'input' : (lambda x: x)}
    | RunnableAssign({'history' : itemgetter('input') | convstore.as_retriever() | long_reorder | docs2str})
    | RunnableAssign({'context' : itemgetter('input') | docstore.as_retriever()  | long_reorder | docs2str})
    | RPrint()
)
stream_chain = chat_prompt | model | StrOutputParser()

def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ##首先根据输入的消息进行检索
    retrieval = retrieval_chain.invoke(message)
    line_buffer = ""

    ## 然后流式传输stream_chain的结果
    for token in stream_chain.stream(retrieval):
        buffer += token
        ## 优化信息打印的格式
        if not return_buffer:
            line_buffer += token
            if "\n" in line_buffer:
                line_buffer = ""
            if ((len(line_buffer)>84 and token and token[0] == " ") or len(line_buffer)>100):
                line_buffer = ""
                yield "\n"
                token = "  " + token.lstrip()
        yield buffer if return_buffer else token

    ##最后将聊天内容保存到对话内存缓冲区中
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)

In [ ]:
import gradio as gr
chatbot = gr.Chatbot(value = [[None, initial_msg]])
demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

try:
    demo.launch(debug=True, share=False, show_api=False, server_port=5000, server_name="0.0.0.0")
    demo.close()
except Exception as e:
    demo.close()
    print(e)
    raise e